In [ ]:
!pip3 install ultralytics
!pip3 install lapx>=0.5.2
!pip3 install --upgrade ultralytics



In [ ]:
from ultralytics import YOLO

In [3]:
model = YOLO('/Users/shravanp/Coding/Robotics/AIScouter/src/models/y11v9.pt')

In [ ]:
results = model.track(source='/Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58.mp4', save=True, tracker='/Users/shravanp/Coding/Robotics/AIScouter/src/config/bytetrack.yaml', persist=True)

In [ ]:
!ffmpeg -i /notebooks/runs/detect/track5/dcmp58.avi -c:v libx264 -crf 23 -preset medium -c:a aac -b:a 128k rb.mp4 -y